In [1]:
import pandas as pd

In [2]:
class Stack:
    def __init__(self):
        self.items = []

    def is_empty(self):
        return len(self.items) == 0

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.is_empty():
            return self.items.pop()
        else:
            raise IndexError("pop from an empty stack")

    def peek(self):
        if not self.is_empty():
            return self.items[-1]
        else:
            raise IndexError("peek from an empty stack")

    def size(self):
        return len(self.items)
    def get_element_at_index(self, index):
        if 0 <= index < len(self.items):
            return self.items[index]
        else:
            raise IndexError("Index out of range")
    def to_string(self):
        return ''.join(self.items)
def reverse_stack(original_stack):
    reversed_stack = Stack()

    while not original_stack.is_empty():
        reversed_stack.push(original_stack.pop())

    return reversed_stack


In [3]:

class DPDA:
    def __init__(self):
        self.states = {'p', 'q', 'qa', 'qb', 'q$'}
        self.input_alphabet = {'a', 'b'}
        self.stack_alphabet = {'a', 'b','S', '$'}
        self.deltaRules = {
            ('p', 'e', 'e'): ('q', 'S'),
            ('q', 'a', 'e'): ('qa', 'e'),
            ('qa', 'e', 'a'): ('q', 'e'),
            ('q', 'b', 'e'): ('qb', 'e'),
            ('q', '$', 'e'): ('q$', 'e'),
            ('qa', 'e', 'S'): ('qa', 'aSb'),
            ('qb', 'e', 'S'): ('qb', 'e')
        }
        self.rRules = {'S->aSb', 'S->e'}
        self.start_state = 'p'
        self.accept_state = 'qa'
        self.reject_state = 'qb'
        self.finish_state = 'q$'
        self.current_state = self.start_state
        self.stack = Stack()
        self.inputStack = Stack()

    def process_input(self, input_string):
        #get columns
        columns = ['step', 'state', 'Unread input', 'Top of stack', '∆ rule used', 'R rule used']
        df = pd.DataFrame(columns=columns)
        rRules_list = list(self.rRules)
        i = 0
        #Setup when i is equal to zero
        df.at[i, 'step'] = i
        df.at[0, 'state'] = self.start_state
        df.at[0, 'Unread input'] = input_string
        df.at[0, 'Top of stack'] = 'e'
        df.at[0, '∆ rule used'] = ''
        df.at[0, 'R rule used'] = ''
        
        #parse through string
        
        self.stack.push('e')
        
        #make a stack of the string
        for char in input_string:
            self.inputStack.push(char)
        
        self.inputStack = reverse_stack(self.inputStack)
        stackLength = self.stack.size()
        while(stackLength > 0):
            if(df.at[i, 'state'] == 'p'and self.stack.peek()=='e'): #first rule -> 2nd rule
                #rule 0
                df.at[i, 'step'] = i
                df.at[i+1, 'state'] = 'q'
                self.stack.pop()
                self.stack.push('S')
                i+=1
                
                #rule 1
                df.at[i, 'R rule used'] = ''
                df.at[i, '∆ rule used'] = '1'
                df.at[i, 'step'] = i
                df.at[i, 'Top of stack'] = self.stack.to_string()[::-1]
                df.at[i, 'Unread input'] = input_string
                
                
                
            if(df.at[i, 'state']=='q' and self.inputStack.peek()=='a'): #second rule
                i+=1
                df.at[i, 'Top of stack'] = self.stack.to_string()[::-1]
                input_string = input_string[1:]
                df.at[i, 'Unread input'] = input_string
                df.at[i, 'R rule used'] = ''
                df.at[i, '∆ rule used'] = '2'
                df.at[i, 'step'] = i
                df.at[i, 'state'] = 'qa'
                df.at[i+1, 'state'] = 'qa'
                
                self.inputStack.pop()
                
                
            if(df.at[i, 'state']=='qa' and self.stack.peek()=='a'):#third rule -> 4th rule
                i+=1
                df.at[i, 'step'] = i
                df.at[i, 'Unread input'] = input_string
                df.at[i, '∆ rule used'] = '3'
                df.at[i, 'R rule used'] = ''
                self.stack.pop()
                df.at[i, 'Top of stack'] = self.stack.to_string()[::-1]
                df.at[i, 'state']='q'
            if(df.at[i, 'state']=='q' and self.inputStack.peek()=='b'): #fourth rule
                i+=1
                df.at[i, '∆ rule used'] = '4'
                df.at[i, 'step'] = i
                input_string = input_string[1:]
                df.at[i, 'Unread input'] = input_string
                df.at[i, 'Top of stack'] = self.stack.to_string()[::-1]
                df.at[i, 'R rule used'] = ''
                df.at[i, 'state'] = 'qb'
            if(df.at[i, 'state']=='qb' and self.stack.peek()== 'b'): #fifth rule
                i+=1
                df.at[i, '∆ rule used'] = '5'
                df.at[i, 'step'] = i
                df.at[i, 'Unread input'] = input_string
                if(self.inputStack.size()==2):
                    self.stack.pop()
                    self.stack.push('e')
                if(self.stack.size()>=2):
                    self.stack.pop()
                df.at[i, 'R rule used'] = ''
                df.at[i, 'state'] = 'q'
                self.inputStack.pop()
                df.at[i, 'Top of stack'] = self.stack.to_string()[::-1]
            if((df.at[i, 'state']=='q' and self.inputStack.peek()== '$') or self.inputStack.peek()== 'e'): #6th rule
                i+=1
                df.at[i, '∆ rule used'] = '6'
                df.at[i, 'step'] = i
                self.inputStack.pop()
                self.inputStack.push('e')
                df.at[i, 'Unread input'] = 'e'
                df.at[i, 'Top of stack'] = 'e'
                df.at[i, 'R rule used'] = ''
                df.at[i, 'state'] = 'q$'
                self.stack.pop()
                stackLength -=1
                self.inputStack.items
                print(df)
            if(df.at[i, 'state']=='qa' and self.stack.peek()=='S'):#7th Rule
                i+=1
                df.at[i, '∆ rule used'] = '7'
                df.at[i+1, 'state'] = 'q'
                df.at[i, 'step'] = i
                df.at[i, 'Unread input'] = input_string
                df.at[i, 'R rule used'] = 's -> aSb'
                self.stack.pop()
                self.stack.push('b')
                self.stack.push('S')
                self.stack.push('a')
                df.at[i, 'Top of stack'] = self.stack.to_string()[::-1]
            if(df.at[i, 'state'] == 'qb'and self.stack.peek()=='S'): #8th Rule
                i+=1
                df.at[i, '∆ rule used'] = '8'
                self.stack.pop()
                df.at[i, 'Top of stack'] = self.stack.to_string()[::-1]
                df.at[i, 'state'] = 'qb'
                df.at[i, 'step'] = i
                df.at[i, 'Unread input'] = input_string
                df.at[i, 'R rule used'] = 's -> e'
                df.at[i+1, 'state'] = 'q'
                
                
        
        #return True
        if(stackLength == 0):
            return True
        else:
            return False
        

In [4]:
dpda = DPDA()

# Test
test_string = input("Enter a test string: ")
result = dpda.process_input(test_string)
if(result == True):
    print(test_string + " is a member of this DPDA")
else:
    print(test_string + " is not a member of this DPDA")

Enter a test string: aaaaaaaaaabbbbbbbbbb$
   step state           Unread input  Top of stack ∆ rule used R rule used
0     0     p  aaaaaaaaaabbbbbbbbbb$             e                        
1     1     q  aaaaaaaaaabbbbbbbbbb$             S           1            
2     2    qa   aaaaaaaaabbbbbbbbbb$             S           2            
3     3    qa   aaaaaaaaabbbbbbbbbb$           aSb           7    s -> aSb
4     4     q   aaaaaaaaabbbbbbbbbb$            Sb           3            
5     5    qa    aaaaaaaabbbbbbbbbb$            Sb           2            
6     6    qa    aaaaaaaabbbbbbbbbb$          aSbb           7    s -> aSb
7     7     q    aaaaaaaabbbbbbbbbb$           Sbb           3            
8     8    qa     aaaaaaabbbbbbbbbb$           Sbb           2            
9     9    qa     aaaaaaabbbbbbbbbb$         aSbbb           7    s -> aSb
10   10     q     aaaaaaabbbbbbbbbb$          Sbbb           3            
11   11    qa      aaaaaabbbbbbbbbb$          Sbbb       